# Processed LIDC data can be found at: https://drive.google.com/drive/folders/1TLpPvR_9hfNdUbD9dFIXNpJ7m50VmD19?usp=sharing

In [1]:
!pip install -r requirements.txt/

     |████████████████████████████████| 377.0 MB 45 kB/s s eta 0:00:01   |▉                               | 10.4 MB 5.2 MB/s eta 0:01:11     |█▏                              | 13.4 MB 5.2 MB/s eta 0:01:11     |██████████████████▎             | 215.1 MB 52.8 MB/s eta 0:00:04     |███████████████████▏            | 226.2 MB 52.8 MB/s eta 0:00:03     |██████████████████████████▎     | 310.2 MB 60.9 MB/s eta 0:00:02
     |████████████████████████████████| 734.6 MB 41 kB/s s eta 0:00:01     |██████████████                  | 319.8 MB 42.8 MB/s eta 0:00:10     |███████████████████████▉        | 547.5 MB 62.0 MB/s eta 0:00:04
     |████████████████████████████████| 488 kB 33.8 MB/s eta 0:00:01
     |████████████████████████████████| 57 kB 10.2 MB/s eta 0:00:01
     |████████████████████████████████| 3.1 MB 52.9 MB/s eta 0:00:01


  Created wheel for wrapt: filename=wrapt-1.12.1-cp36-cp36m-linux_x86_64.whl size=66149 sha256=57da50aebfd5c80a897443f54fbae2c12d0a65019eb44be85f95e6cd2d9a7e93
  Stored in directory: /home/zongwei/.cache/pip/wheels/32/42/7f/23cae9ff6ef66798d00dc5d659088e57dbba01566f6c60db63
Successfully built wrapt
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 1.13.0
    Uninstalling tensorflow-estimator-1.13.0:
      Successfully uninstalled tensorflow-estimator-1.13.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 1.13.1
    Uninstalling tensorboard-1.13.1:
      Successfully uninstalled tensorboard-1.13.1
ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

tensorflow 1.13.1 r

In [2]:
#!/usr/bin/env python
# coding: utf-8
from __future__ import print_function
import warnings
warnings.filterwarnings('ignore')
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # or any {'0', '1', '2'}
import tensorflow as tf
print("keras = {}".format(tf.keras.__version__))
print("tensorflow-gpu = {}".format(tf.__version__))
try:
    tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
except:
    pass
import random
import shutil
import argparse
import sklearn
from pathlib import Path
from utils import *
from unet3d import *
from config import *
import BraTS

class set_args():
    gpu = 0
    data = None
    apps = 'bms'
    run = 1
    cv = None
    subsetting = None
    suffix = 'random'
    task = 'segmentation'
    
args = set_args()

if args.gpu is not None:
    os.environ["CUDA_VISIBLE_DEVICES"] = str(args.gpu)

conf = bms_config(args)

# If keras error just reinstall it

keras = 2.2.4-tf
tensorflow-gpu = 1.14.0


Using TensorFlow backend.


In [7]:
dataset = BraTS.DataSet(
    brats_root='/home/ioanna/PycharmProjects/uva-thesis/data',
    year=2017)
x_train = dataset.train

lol = x_train._mris[2]

print(x_train.mris.shape, x_train.segs.shape)

import matplotlib.pyplot as plt
plt.imshow(x_train.subset(['Brats17_TCIA_630_1']).mris[:,1,:,:,90].squeeze())

# x_train, y_train = load_image(conf, 'train')
# print('x_train: {} | {} ~ {}'.format(x_train.shape, np.min(x_train), np.max(x_train)))
# print('y_train: {} | {} ~ {}'.format(y_train.shape, np.min(y_train), np.max(y_train)))
# 
# x_valid, y_valid = load_image(conf, 'valid')
# print('x_valid: {} | {} ~ {}'.format(x_valid.shape, np.min(x_valid), np.max(x_valid)))
# print('y_valid: {} | {} ~ {}'.format(y_valid.shape, np.min(y_valid), np.max(y_valid)))
# 
# x_test, y_test = load_image(conf, 'test')
# print('x_test: {} | {} ~ {}'.format(x_test.shape, np.min(x_test), np.max(x_test)))
# print('y_test: {} | {} ~ {}'.format(y_test.shape, np.min(y_test), np.max(y_test)))import tensorflow.keras as keras


MemoryError: Unable to allocate 75.8 GiB for an array with shape (285, 4, 240, 240, 155) and data type float64

# Fine-tune Models Genesis

In [ ]:
args.suffix = 'genesis'
conf = bms_config(args)
conf.display()

### Train

In [ ]:
print(1)
model = unet_model_3d((1,conf.input_rows,conf.input_cols,conf.input_deps), batch_normalization=True)
if conf.weights is not None:
    print("[INFO] Load pre-trained weights from {}".format(conf.weights))
    model.load_weights(conf.weights)
print(2)
model, callbacks = model_setup(model, conf, task=args.task)
print(3)
while conf.batch_size > 1:
    # To find a largest batch size that can be fit into GPU
    print(conf.batch_size)
    try:
        model.fit_generator(x_train, y_train,
                  validation_data=(x_valid, y_valid),
                  batch_size=conf.batch_size,
                  epochs=conf.nb_epoch, 
                  verbose=conf.verbose, 
                  shuffle=True,
                  callbacks=callbacks)
        break
    except tf.errors.ResourceExhaustedError as e:
        conf.batch_size = int(conf.batch_size - 2)
        print("\n> Batch size = {}".format(conf.batch_size))

### Test

In [ ]:
model = unet_model_3d((1,conf.input_rows,conf.input_cols,conf.input_deps), batch_normalization=True)
print("[INFO] Load trained model from {}".format( os.path.join(conf.model_path, conf.exp_name+".h5") ))
model.load_weights( os.path.join(conf.model_path, conf.exp_name+".h5") )

p_test = segmentation_model_evaluation(model=model, config=conf, x=x_test, y=y_test, note=conf.exp_name)

### Visualization

In [ ]:
p_test = np.squeeze(p_test)
for i in range(0, x_test.shape[0], 80):
    plot_image_truth_prediction(x_test[i], y_test[i], p_test[i], rows=5, cols=5)

# Train from scratch

In [ ]:
args.suffix = 'random'
conf = ncs_config(args)
conf.display()

### Train

In [ ]:
model = unet_model_3d((1,conf.input_rows,conf.input_cols,conf.input_deps), batch_normalization=True)
if conf.weights is not None:
    print("[INFO] Load pre-trained weights from {}".format(conf.weights))
    model.load_weights(conf.weights)
model, callbacks = model_setup(model, conf, task=args.task)

while conf.batch_size > 1:
    # To find a largest batch size that can be fit into GPU
    try:
        model.fit(x_train, y_train,
                  validation_data=(x_valid, y_valid),
                  batch_size=conf.batch_size,
                  epochs=conf.nb_epoch, 
                  verbose=conf.verbose, 
                  shuffle=True,
                  callbacks=callbacks)
        break
    except tf.errors.ResourceExhaustedError as e:
        conf.batch_size = int(conf.batch_size - 2)
        print("\n> Batch size = {}".format(conf.batch_size))

### Test

In [ ]:
model = unet_model_3d((1,conf.input_rows,conf.input_cols,conf.input_deps), batch_normalization=True)
print("[INFO] Load trained model from {}".format( os.path.join(conf.model_path, conf.exp_name+".h5") ))
model.load_weights( os.path.join(conf.model_path, conf.exp_name+".h5") )

p_test = segmentation_model_evaluation(model=model, config=conf, x=x_test, y=y_test, note=conf.exp_name)

### Visualization

In [ ]:
p_test = np.squeeze(p_test)
for i in range(0, x_test.shape[0], 80):
    plot_image_truth_prediction(x_test[i], y_test[i], p_test[i], rows=5, cols=5)